### Compiles and Outputs Step 1 Confusion Matrix

1. Confusion matrix for each subject (output: confusion_matrix.csv)
2. Group average confusion matrix (output: confusion_matrix_all.csv)
3. Average confusion matrix by group - Meditators vs. Controls (output: confusion_matrix_byGroup.csv)

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import glob
import fnmatch

In [ ]:
root_dir = '/Path/to/EMBODY/files'

In [ ]:
# define analyses, subject IDs and groups
ml_brain = ['phase1_demo']
subjects = [124, 132]

# Enter subject numbers
Meditators = [124]
Controls = [132]

In [ ]:
def condition_decisions(condition_number):
    
    # Compile decision counts
    decision_count=[0, 0, 0, 0, 0]
    
    for index in range(len(condition_list)):
        current_value = condition_list[index]
        decision_value = decision_list[index]
        
        # sums decisions from breath condition
        if current_value == condition_number:
            decision_count[decision_value - 1] += 1
    
    return decision_count

In [ ]:
# 1. Compiles step 1 confusion matrix for each subj
# Make new directories before running
# 5x5 matrix of classifier decisions: 1) for each subj, 2) average of subjs, 3) average of groups
# 3x3 matrix of categories of most interest: Breath, Stop, Self
# Calculation: for each instructed condition, # of decisions in each category (5x5)

for analysis in ml_brain:
    
    for subj in subjects:
        
        os.chdir('%s/%s/results/step1/%d/' % (root_dir, analysis, subj))
        
        data = sio.loadmat('pretty_results_step1.mat')['prettyResult'][0][0]
        condition = data[4]
        decision = data[6]
        condition_list = list(condition[0])
        decision_list = list(decision[0])

        breath = condition_decisions(1)
        feet = condition_decisions(2)
        stop = condition_decisions(3)
        self = condition_decisions(4)
        sounds = condition_decisions(5)

        confusion_matrix = np.stack((breath, feet, stop, self, sounds))

        index_list = ['breath_cond', 'feet_cond', 'stop_cond', 'self_cond', 'sounds_cond']
        columns_list = ['breath_dec', 'feet_dec', 'stop_dec', 'self_dec', 'sounds_dec']

        confusion_matrix_df = pd.DataFrame(confusion_matrix, index = index_list, columns = columns_list) # convert into a pandas dataframe
        confusion_matrix_df.to_csv('confusion_matrix.csv') # writes out data to current dir

In [ ]:
## 2. Compile group average confusion matrix

for analysis in ml_brain:
    
    appended_data = []
    for infile in glob.glob('%s/%s/results/step1/*/confusion_matrix.csv' % (root_dir, analysis)):
        data = pd.read_csv(infile, sep=',', index_col=0)
        # store DataFrame in list
        appended_data.append(data)
    
    # Concatenate confusion matrices, compute mean
    appended_data = pd.concat(appended_data, axis=0) # concat by rows
    confusion_matrix_mean = appended_data.groupby(level=0).mean() # compute mean by rows
    df = pd.DataFrame(confusion_matrix_mean)
    confusion_matrix_mean_df = df.reindex(['breath_cond', 'feet_cond', 'stop_cond', 'self_cond', 'sounds_cond'])
    
    print(confusion_matrix_mean_df)

In [ ]:
## 3. Compute confusion matrix by group (Meditators vs. Controls)

for analysis in ml_brain:
    # Meditators
    meditator_data = []
    for subj in Meditators:
        
        os.chdir('%s/%s/results/step1/%d/' % (root_dir, analysis, subj))
        infile = 'confusion_matrix.csv'
        data = pd.read_csv(infile, sep=',', index_col=0)
        # store DataFrame in list
        meditator_data.append(data)

    # Concatenate confusion matrices
    meditator_data = pd.concat(meditator_data, axis=0) # concat by rows, 80 rows x 5 columns
    meditator_mean = meditator_data.groupby(level=0).mean() # mean by rows, stop 5th instead of 3rd row
    df = pd.DataFrame(meditator_mean) # convert into a pandas dataframe
    meditator_mean_df = df.reindex(['breath_cond', 'feet_cond', 'stop_cond', 'self_cond', 'sounds_cond'])
    #print meditator_mean_df
    
    # write DataFrame csv
    os.chdir('%s/%s/results/step1_compile/' % (root_dir, analysis))
    meditator_mean_df.to_csv('step1_2_confusion_matrix_meditator_mean.csv')
    
    # Controls
    control_data = []
    for subj in Controls:
        os.chdir('%s/%s/results/step1/%d/' % (root_dir, analysis, subj))
        infile = 'confusion_matrix.csv'
        data = pd.read_csv(infile, sep=',', index_col=0)
        # store DataFrame in list
        control_data.append(data)

    # Concatenate confusion matrices
    control_data = pd.concat(control_data, axis=0) # concat by rows, 80 rows x 5 columns
    control_mean = control_data.groupby(level=0).mean() # mean by rows, stop 5th instead of 3rd row
    df = pd.DataFrame(control_mean) # convert into a pandas dataframe
    control_mean_df = df.reindex(['breath_cond', 'feet_cond', 'stop_cond', 'self_cond', 'sounds_cond'])
    #print control_mean_df
    
    # write DataFrame csv
    os.chdir('%s/%s/results/step1_compile/' % (root_dir, analysis))
    control_mean_df.to_csv('step1_2_confusion_matrix_control_mean.csv')